# Hands-on: Training and deploying Question Answering with BERT

Pre-trained language representations have been shown to improve many downstream NLP tasks such as question answering, and natural language inference. Devlin, Jacob, et al proposed BERT [1] (Bidirectional Encoder Representations from Transformers), which fine-tunes deep bidirectional representations on a wide range of tasks with minimal task-specific parameters, and obtained state- of-the-art results.

In this tutorial, we will focus on adapting the BERT model for the question answering task on the SQuAD dataset. Specifically, we will:

- understand how to pre-process the SQuAD dataset to leverage the learnt representation in BERT,
- adapt the BERT model to the question answering task, and
- load a trained model to perform inference on the SQuAD dataset

In [1]:
# this notebook requires mxnet-cu101 >= 1.6.0b20191102, gluonnlp >= 0.8.1
# we can create a sagemaker notebook instance with the lifecycle configuration file: sagemaker-lifecycle.config
!pip list | grep mxnet
!pip list | grep gluonnlp

keras-mxnet                        2.2.4.2       
mxnet-cu101                        1.6.0b20191122
mxnet-model-server                 1.0.5         
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
gluonnlp                           0.9.0.dev0    
You are using pip version 10.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


## Load MXNet and GluonNLP

We first import the libraries:

In [2]:
import argparse, collections, time, logging
import json
import os
import io
import copy
import random
import warnings

import numpy as np
import gluonnlp as nlp
import mxnet as mx
import bert
import qa_utils

from gluonnlp.data import SQuAD
from bert.model.qa import BertForQALoss, BertForQA
from bert.data.qa import SQuADTransform, preprocess_dataset
from bert.bert_qa_evaluate import get_F1_EM, predict, PredResult

# Hyperparameters
parser = argparse.ArgumentParser('BERT finetuning')
parser.add_argument('--epochs', type=int, default=3)
parser.add_argument('--batch_size', default=32)
parser.add_argument('--num_epochs', default=1)
parser.add_argument('--lr', default=5e-5)

parser.add_argument('--output_dir',
                    type=str,
                    default='./output_dir',
                    help='The output directory where the model params will be written.'
                    ' default is ./output_dir')
parser.add_argument('--test_batch_size',
                    type=int,
                    default=24,
                    help='Test batch size. default is 24')
parser.add_argument('--optimizer',
                    type=str,
                    default='bertadam',
                    help='optimization algorithm. default is bertadam')
parser.add_argument('--accumulate',
                    type=int,
                    default=None,
                    help='The number of batches for '
                    'gradients accumulation to simulate large batch size. Default is None')
parser.add_argument('--warmup_ratio',
                    type=float,
                    default=0.1,
                    help='ratio of warmup steps that linearly increase learning rate from '
                    '0 to target learning rate. default is 0.1')
parser.add_argument('--log_interval',
                    type=int,
                    default=50,
                    help='report interval. default is 50')
parser.add_argument('--max_seq_length',
                    type=int,
                    default=384,
                    help='The maximum total input sequence length after WordPiece tokenization.'
                    'Sequences longer than this will be truncated, and sequences shorter '
                    'than this will be padded. default is 384')
parser.add_argument('--doc_stride',
                    type=int,
                    default=128,
                    help='When splitting up a long document into chunks, how much stride to '
                    'take between chunks. default is 128')
parser.add_argument('--max_query_length',
                    type=int,
                    default=64,
                    help='The maximum number of tokens for the question. Questions longer than '
                    'this will be truncated to this length. default is 64')
parser.add_argument('--n_best_size',
                    type=int,
                    default=20,
                    help='The total number of n-best predictions to generate in the '
                    'nbest_predictions.json output file. default is 20')
parser.add_argument('--max_answer_length',
                    type=int,
                    default=30,
                    help='The maximum length of an answer that can be generated. This is needed '
                    'because the start and end predictions are not conditioned on one another.'
                    ' default is 30')
# parser.add_argument('--version_2',
#                     action='store_true',
#                     help='SQuAD examples whether contain some that do not have an answer.')
parser.add_argument('--null_score_diff_threshold',
                    type=float,
                    default=0.0,
                    help='If null_score - best_non_null is greater than the threshold predict null.'
                    'Typical values are between -1.0 and -5.0. default is 0.0')
parser.add_argument('--sentencepiece',
                    type=str,
                    default=None,
                    help='Path to the sentencepiece .model file for both tokenization and vocab.')
# parser.add_argument('--debug',
#                     action='store_true',
#                     help='Run the example in test mode for sanity checks')


args = parser.parse_args([])


epochs = args.epochs
batch_size = args.batch_size
num_epochs = args.num_epochs
lr = args.lr

output_dir = args.output_dir
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
test_batch_size = args.test_batch_size
optimizer = args.optimizer
accumulate = args.accumulate
warmup_ratio = args.warmup_ratio
log_interval = args.log_interval
max_seq_length = args.max_seq_length
doc_stride = args.doc_stride
max_query_length = args.max_query_length
n_best_size = args.n_best_size

## Inspect the SQuAD Dataset

Then we take a look at the Stanford Question Answering Dataset (SQuAD). The dataset can be downloaded using the `nlp.data.SQuAD` API. In this tutorial, we create a small dataset with 3 samples from the SQuAD dataset for demonstration purpose.

The question answering task on the SQuAD dataset is setup the following way. For each sample in the dataset, a context is provided. The context is usually a long paragraph which contains lots of information. Then a question asked based on the context. The goal is to find the text span in the context that answers the question in the sample.

In [3]:
full_data = nlp.data.SQuAD(segment='dev', version='1.1')
# loading a subset of the dev set of SQuAD
num_target_samples = 3
target_samples = [full_data[i] for i in range(num_target_samples)]
dataset = mx.gluon.data.SimpleDataset(target_samples)
print('Number of samples in the created dataset subsampled from SQuAD = %d'%len(dataset))

Number of samples in the created dataset subsampled from SQuAD = 3


Let's take a look at a sample from the dataset. In this sample, the question is about the location of the game, with a description about the Super Bowl 50 game as the context. Note that three different answer spans are correct for this question, and they start from index 403, 355 and 355 in the context respectively.

In [4]:
sample = dataset[2]

context_idx = 3

print('\nContext:\n')
print(sample[context_idx])


Context:

Super Bowl 50 was an American football game to determine the champion of the National Football League (NFL) for the 2015 season. The American Football Conference (AFC) champion Denver Broncos defeated the National Football Conference (NFC) champion Carolina Panthers 24–10 to earn their third Super Bowl title. The game was played on February 7, 2016, at Levi's Stadium in the San Francisco Bay Area at Santa Clara, California. As this was the 50th Super Bowl, the league emphasized the "golden anniversary" with various gold-themed initiatives, as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals (under which the game would have been known as "Super Bowl L"), so that the logo could prominently feature the Arabic numerals 50.


In [5]:
question_idx = 2
answer_idx = 4
answer_pos_idx = 5

print("\nQuestion")
print(sample[question_idx])
print("\nCorrect Answer Spans")
print(sample[answer_idx])
print("\nAnswer Span Start Indices:")
print(sample[answer_pos_idx])


Question
Where did Super Bowl 50 take place?

Correct Answer Spans
['Santa Clara, California', "Levi's Stadium", "Levi's Stadium in the San Francisco Bay Area at Santa Clara, California."]

Answer Span Start Indices:
[403, 355, 355]


## Data Pre-processing for QA with BERT

Recall that during BERT pre-training, it takes a sentence pair as the input, separated by the 'SEP' special token. For SQuAD, we can feed the context-question pair as the sentence pair input. To use BERT to predict the starting and ending span of the answer, we can add a classification layer for each token in the context texts, to predict if a token is the start or the end of the answer span. 

![qa](natural_language_understanding/qa.png)

In the next few code blocks, we will work on pre-processing the samples in the SQuAD dataset in the desired format with these special separators. 


### Get Pre-trained BERT Model

First, let's use the *get_model* API in GluonNLP to get the model definition for BERT, and the vocabulary used for the BERT model. Note that we discard the pooler and classifier layers used for the next sentence prediction task, as well as the decoder layers for the masked language model task during the BERT pre-training phase. These layers are not useful for predicting the starting and ending indices of the answer span.

The list of pre-trained BERT models available in GluonNLP can be found [here](http://gluon-nlp.mxnet.io/model_zoo/bert/index.html).

In [6]:
bert_model, vocab = nlp.model.get_model('bert_12_768_12',
                                        dataset_name='book_corpus_wiki_en_uncased',
                                        use_classifier=False,
                                        use_decoder=False,
                                        use_pooler=False,
                                        pretrained=False)

Note that there are several special tokens in the vocabulary for BERT. In particular, the `[SEP]` token is used for separating the sentence pairs, and the `[CLS]` token is added at the beginning of the sentence pairs. They will be used to pre-process the SQuAD dataset later.

In [7]:
print(vocab)

Vocab(size=30522, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")


### Tokenization

The second step is to process the samples using the same tokenizer used for BERT, which is provided as the `BERTTokenizer` API in GluonNLP. Note that instead of word level and character level representation, BERT uses subwords to represent a word, separated `##`. 

In the following example, the word `suspending` is tokenized as two subwords (`suspend` and `##ing`), and `numerals` is tokenized as three subwords (`nu`, `##meral`, `##s`).

In [8]:
tokenizer = nlp.data.BERTTokenizer(vocab=vocab, lower=True)

tokenizer("as well as temporarily suspending the tradition of naming each Super Bowl game with Roman numerals")

['as',
 'well',
 'as',
 'temporarily',
 'suspend',
 '##ing',
 'the',
 'tradition',
 'of',
 'naming',
 'each',
 'super',
 'bowl',
 'game',
 'with',
 'roman',
 'nu',
 '##meral',
 '##s']

### Sentence Pair Composition

With the tokenizer inplace, we are ready to process the question-context texts and compose sentence pairs. The functionality is available via the `SQuADTransform` API. 

In [9]:
transform = bert.data.qa.SQuADTransform(tokenizer, is_pad=False, is_training=False, do_lookup=False)
dev_data_transform, _ = bert.data.qa.preprocess_dataset(dataset, transform)
logging.info('The number of examples after preprocessing:{}'.format(len(dev_data_transform)))

Done! Transform dataset costs 0.17 seconds.


Let's take a look at the sample after the transformation:

In [10]:
sample = dev_data_transform[2]
print('\nsegment type: \n' + str(sample[2]))
print('\ntext length: ' + str(sample[3]))
print('\nsentence pair: \n' + str(sample[1]))


segment type: 
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

text length: 168

sentence pair: 
['[CLS]', 'where', 'did', 'super', 'bowl', '50', 'take', 'place', '?', '[SEP]', 'super', 'bowl', '50', 'was', 'an', 'american', 'football', 'game', 'to', 'determine', 'the', 'champion', 'of', 'the', 'national', 'football', 'league', '(', 'nfl', ')', 'for', 'the', '2015', 'season', '.', 'the', 'american', 'football', 'conference', '(', 'afc', ')', 'champion', 'denver', 'broncos', 'defeated', 'the', 'national', 'football', 'conference', '(', 

### Vocabulary Lookup

Finally, we convert the transformed texts to subword indices, which are used to contructor NDArrays as the inputs to the model.

In [11]:
def vocab_lookup(example_id, subwords, type_ids, length, start, end):
    indices = vocab[subwords]
    return example_id, indices, type_ids, length, start, end

dev_data_transform = dev_data_transform.transform(vocab_lookup, lazy=False)
print(dev_data_transform[2][1])

[2, 2073, 2106, 3565, 4605, 2753, 2202, 2173, 1029, 3, 3565, 4605, 2753, 2001, 2019, 2137, 2374, 2208, 2000, 5646, 1996, 3410, 1997, 1996, 2120, 2374, 2223, 1006, 5088, 1007, 2005, 1996, 2325, 2161, 1012, 1996, 2137, 2374, 3034, 1006, 10511, 1007, 3410, 7573, 14169, 3249, 1996, 2120, 2374, 3034, 1006, 22309, 1007, 3410, 3792, 12915, 2484, 1516, 2184, 2000, 7796, 2037, 2353, 3565, 4605, 2516, 1012, 1996, 2208, 2001, 2209, 2006, 2337, 1021, 1010, 2355, 1010, 2012, 11902, 1005, 1055, 3346, 1999, 1996, 2624, 3799, 3016, 2181, 2012, 4203, 10254, 1010, 2662, 1012, 2004, 2023, 2001, 1996, 12951, 3565, 4605, 1010, 1996, 2223, 13155, 1996, 1000, 3585, 5315, 1000, 2007, 2536, 2751, 1011, 11773, 11107, 1010, 2004, 2092, 2004, 8184, 28324, 2075, 1996, 4535, 1997, 10324, 2169, 3565, 4605, 2208, 2007, 3142, 16371, 28990, 2015, 1006, 2104, 2029, 1996, 2208, 2052, 2031, 2042, 2124, 2004, 1000, 3565, 4605, 1048, 1000, 1007, 1010, 2061, 2008, 1996, 8154, 2071, 14500, 3444, 1996, 5640, 16371, 28990, 2015

## Model Definition

After the data is processed, we can define the model that uses the representation produced by BERT for predicting the starting and ending positions of the answer span.

We download a BERT model trained on the SQuAD dataset, prepare the dataloader.

In [12]:
net = BertForQA(bert_model)

ctx = mx.gpu(0)
## multi-gpu training
# GPU_COUNT = 4 # increase if you have more
# ctx = [mx.gpu(i) for i in range(GPU_COUNT)]

ckpt = qa_utils.download_qa_ckpt()
net.load_parameters(ckpt, ctx=ctx)

batch_size = 1
dev_dataloader = mx.gluon.data.DataLoader(
    dev_data_transform, batch_size=batch_size, shuffle=False)

Downloaded checkpoint to ./temp/bert_qa-7eb11865.params


In [13]:
# all_results = collections.defaultdict(list)

# total_num = 0
# for data in dev_dataloader:
#     example_ids, inputs, token_types, valid_length, _, _ = data
#     total_num += len(inputs)
#     batch_size = inputs.shape[0]
#     pred_start, pred_end = net(inputs.astype('float32').as_in_context(ctx),
#                                token_types.astype('float32').as_in_context(ctx),
#                                valid_length.astype('float32').as_in_context(ctx))

#     example_ids = example_ids.asnumpy().tolist()
#     pred_start = pred_start.reshape(batch_size, -1).asnumpy()
#     pred_end = pred_end.reshape(batch_size, -1).asnumpy()
    
#     for example_id, start, end in zip(example_ids, pred_start, pred_end):
#         all_results[example_id].append(PredResult(start=start, end=end))

In [14]:
# qa_utils.predict(dataset, all_results, vocab)

### Let's Train the Model

Now we can put all the pieces together, and start fine-tuning the model with a few epochs.

In [15]:
# net = BertForQA(bert=bert_model)
# nlp.utils.load_parameters(net, pretrained_bert_parameters, ctx=ctx,
#                           ignore_extra=True, cast_dtype=True)
net.span_classifier.initialize(init=mx.init.Normal(0.02), ctx=ctx)
net.hybridize(static_alloc=True)

loss_function = BertForQALoss()
loss_function.hybridize(static_alloc=True)

/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/gluon/parameter.py:887: UserWarning: Parameter 'bertforqa0_dense0_weight' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)
/home/ec2-user/anaconda3/envs/mxnet_p36/lib/python3.6/site-packages/mxnet/gluon/parameter.py:887: UserWarning: Parameter 'bertforqa0_dense0_bias' is already initialized, ignoring. Set force_reinit=True to re-initialize.
  v.initialize(None, ctx, init, force_reinit=force_reinit)


In [16]:

batchify_fn = nlp.data.batchify.Tuple(
    nlp.data.batchify.Stack(),
    nlp.data.batchify.Pad(axis=0, pad_val=vocab[vocab.padding_token]),
    nlp.data.batchify.Pad(axis=0, pad_val=vocab[vocab.padding_token]),
    nlp.data.batchify.Stack('float32'),
    nlp.data.batchify.Stack('float32'),
    nlp.data.batchify.Stack('float32'))

np.random.seed(6)
random.seed(6)
mx.random.seed(6)

log = logging.getLogger('gluonnlp')
log.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    fmt='%(levelname)s:%(name)s:%(asctime)s %(message)s', datefmt='%H:%M:%S')


segment = 'train'  # if not args.debug else 'dev'
log.info('Loading %s data...', segment)
#     if version_2:
#         train_data = SQuAD(segment, version='2.0')
#     else:
train_data = SQuAD(segment, version='1.1')
#     if args.debug:
#         sampled_data = [train_data[i] for i in range(1000)]
#         train_data = mx.gluon.data.SimpleDataset(sampled_data)
log.info('Number of records in Train data:{}'.format(len(train_data)))

train_data_transform, _ = preprocess_dataset(
    train_data, SQuADTransform(
        copy.copy(tokenizer),
        max_seq_length=max_seq_length,
        doc_stride=doc_stride,
        max_query_length=max_query_length,
        is_pad=True,
        is_training=True))
log.info('The number of examples after preprocessing:{}'.format(
    len(train_data_transform)))

train_dataloader = mx.gluon.data.DataLoader(
    train_data_transform, batchify_fn=batchify_fn,
    batch_size=batch_size, num_workers=4, shuffle=True)

INFO:gluonnlp:Loading train data...
INFO:gluonnlp:Number of records in Train data:87599
INFO:gluonnlp:The number of examples after preprocessing:88641


Done! Transform dataset costs 57.50 seconds.


In [ ]:

def train(log, train_data_transform, train_dataloader):
    """Training function."""

    log.info('Start Training')

    optimizer_params = {'learning_rate': lr}
    trainer = mx.gluon.Trainer(net.collect_params(), optimizer,
                               optimizer_params, update_on_kvstore=False)

    num_train_examples = len(train_data_transform)
    step_size = batch_size * accumulate if accumulate else batch_size
    num_train_steps = int(num_train_examples / step_size * epochs)
    num_warmup_steps = int(num_train_steps * warmup_ratio)
    step_num = 0
    
    def set_new_lr(step_num, batch_id):
        """set new learning rate"""
        # set grad to zero for gradient accumulation
        if accumulate:
            if batch_id % accumulate == 0:
                net.collect_params().zero_grad()
                step_num += 1
        else:
            step_num += 1
        # learning rate schedule
        # Notice that this learning rate scheduler is adapted from traditional linear learning
        # rate scheduler where step_num >= num_warmup_steps, new_lr = 1 - step_num/num_train_steps
        if step_num < num_warmup_steps:
            new_lr = lr * step_num / num_warmup_steps
        else:
            offset = (step_num - num_warmup_steps) * lr / \
                (num_train_steps - num_warmup_steps)
            new_lr = lr - offset
        trainer.set_learning_rate(new_lr)
        return step_num

    # Do not apply weight decay on LayerNorm and bias terms
    for _, v in net.collect_params('.*beta|.*gamma|.*bias').items():
        v.wd_mult = 0.0
    # Collect differentiable parameters
    params = [p for p in net.collect_params().values()
              if p.grad_req != 'null']
    # Set grad_req if gradient accumulation is required
    if accumulate:
        for p in params:
            p.grad_req = 'add'

    epoch_tic = time.time()
    total_num = 0
    log_num = 0
    for epoch_id in range(epochs):
        step_loss = 0.0
        tic = time.time()
        for batch_id, data in enumerate(train_dataloader):
            # set new lr
            step_num = set_new_lr(step_num, batch_id)
            # forward and backward
            with mx.autograd.record():
                _, inputs, token_types, valid_length, start_label, end_label = data

                log_num += len(inputs)
                total_num += len(inputs)

                out = net(inputs.astype('float32').as_in_context(ctx),
                          token_types.astype('float32').as_in_context(ctx),
                          valid_length.astype('float32').as_in_context(ctx))

                ls = loss_function(out, [
                    start_label.astype('float32').as_in_context(ctx),
                    end_label.astype('float32').as_in_context(ctx)]).mean()

                if accumulate:
                    ls = ls / accumulate
            ls.backward()
            # update
            if not accumulate or (batch_id + 1) % accumulate == 0:
                trainer.allreduce_grads()
                nlp.utils.clip_grad_global_norm(params, 1)
                trainer.update(1)

            step_loss += ls.asscalar()
        
#         for batch_id, data in enumerate(train_dataloader):
#             # set new lr
#             step_num = set_new_lr(step_num, batch_id)
#             # forward and backward
# #             with mx.autograd.record():
#             _, inputs, token_types, valid_length, start_label, end_label = data

#             log_num += len(inputs)
#             total_num += len(inputs)

#             def split_and_load(data, ctx):
#                 n, k = data.shape[0], len(ctx)
#                 print(n, k)
#                 if (n//k)*k != n:
#                     drop = n - (n//k)*k
#                     data = data[:-drop]
#                     n, k = data.shape[0], len(ctx)
#                 assert (n//k)*k == n, '# examples is not divided by # devices'
#                 idx = list(range(0, n+1, n//k))
#                 return [data[idx[i]:idx[i+1]].as_in_context(ctx[i]) for i in range(k)]
            
# #                 def train_batch(inputs, params, ctx, lr):
#                     # split the data batch and load them on GPUs
#             print(len(inputs[0]), len(token_types[0]), len(valid_length[0]), len(start_label[0]), len(end_label[0]))
#             inputs = split_and_load(inputs[0], ctx)
#             token_types = split_and_load(token_types[0], ctx)
# #             valid_length = split_and_load(valid_length, ctx)
# #             start_label = split_and_load(start_label[0], ctx)
# #             end_label = split_and_load(end_label[0], ctx)

#             # run forward on each GPU
#             with mx.autograd.record():
#                 losses = [loss_function(net(X, Y, W), [U, V])
#                           for X, Y, W, U, V in zip(inputs, token_types, valid_length, start_label, end_label)]
#             # run backward on each gpu
#             for ls in losses:
#                 ls.backward()
#                 step_loss += ls.asscalar()
#             # aggregate gradient over GPUs
#             for i in range(len(params[0])):
#                 allreduce([params[c][i].grad for c in range(len(ctx))])
#             # update parameters with SGD on each GPU
#             for p in params:
#                 nlp.utils.clip_grad_global_norm(P, 1)
#                 trainer.update(1)
            if (batch_id + 1) % log_interval == 0: 
                toc = time.time()
                log.info('Epoch: {}, Batch: {}/{}, Loss={:.4f}, lr={:.7f} Time cost={:.1f} Thoughput={:.2f} samples/s'  # pylint: disable=line-too-long
                         .format(epoch_id, batch_id, len(train_dataloader),
                                 step_loss / log_interval,
                                 trainer.learning_rate, toc - tic, log_num/(toc - tic)))
                tic = time.time()
                step_loss = 0.0
                log_num = 0
        epoch_toc = time.time()
        log.info('Time cost={:.2f} s, Thoughput={:.2f} samples/s'.format(
            epoch_toc - epoch_tic, total_num/(epoch_toc - epoch_tic)))

    net.save_parameters(os.path.join(output_dir, 'net.params'))


train(log, train_data_transform, train_dataloader)

INFO:gluonnlp:Start Training
INFO:gluonnlp:Epoch: 0, Batch: 49/88641, Loss=0.6069, lr=0.0000001 Time cost=10.9 Thoughput=4.59 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 99/88641, Loss=0.6334, lr=0.0000002 Time cost=4.1 Thoughput=12.11 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 149/88641, Loss=0.7261, lr=0.0000003 Time cost=4.1 Thoughput=12.06 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 199/88641, Loss=0.5254, lr=0.0000004 Time cost=4.3 Thoughput=11.62 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 249/88641, Loss=0.5394, lr=0.0000005 Time cost=4.1 Thoughput=12.08 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 299/88641, Loss=0.4510, lr=0.0000006 Time cost=4.1 Thoughput=12.27 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 349/88641, Loss=0.7645, lr=0.0000007 Time cost=4.1 Thoughput=12.25 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 399/88641, Loss=0.6211, lr=0.0000008 Time cost=4.1 Thoughput=12.24 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 449/88641, Loss=0.6223, lr=0.0000008 Time cost=4.1 Thoughput=12.25 samples/s
I

INFO:gluonnlp:Epoch: 0, Batch: 3849/88641, Loss=1.1156, lr=0.0000072 Time cost=4.1 Thoughput=12.27 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 3899/88641, Loss=0.9664, lr=0.0000073 Time cost=4.1 Thoughput=12.13 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 3949/88641, Loss=0.4874, lr=0.0000074 Time cost=4.1 Thoughput=12.09 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 3999/88641, Loss=0.7005, lr=0.0000075 Time cost=4.1 Thoughput=12.17 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 4049/88641, Loss=1.0819, lr=0.0000076 Time cost=4.1 Thoughput=12.20 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 4099/88641, Loss=0.6048, lr=0.0000077 Time cost=4.1 Thoughput=12.14 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 4149/88641, Loss=0.8377, lr=0.0000078 Time cost=4.1 Thoughput=12.15 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 4199/88641, Loss=0.6581, lr=0.0000079 Time cost=4.1 Thoughput=12.27 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 4249/88641, Loss=0.9878, lr=0.0000080 Time cost=4.1 Thoughput=12.23 samples/s
INFO:gluonnlp:Epoch

INFO:gluonnlp:Epoch: 0, Batch: 7649/88641, Loss=1.2989, lr=0.0000144 Time cost=4.1 Thoughput=12.10 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 7699/88641, Loss=0.7474, lr=0.0000145 Time cost=4.1 Thoughput=12.24 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 7749/88641, Loss=0.8412, lr=0.0000146 Time cost=4.1 Thoughput=12.25 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 7799/88641, Loss=1.1350, lr=0.0000147 Time cost=4.1 Thoughput=12.25 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 7849/88641, Loss=0.9285, lr=0.0000148 Time cost=4.1 Thoughput=12.25 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 7899/88641, Loss=1.0477, lr=0.0000149 Time cost=4.2 Thoughput=11.99 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 7949/88641, Loss=0.8970, lr=0.0000149 Time cost=4.1 Thoughput=12.27 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 7999/88641, Loss=0.9430, lr=0.0000150 Time cost=4.1 Thoughput=12.30 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 8049/88641, Loss=0.8372, lr=0.0000151 Time cost=4.1 Thoughput=12.30 samples/s
INFO:gluonnlp:Epoch

INFO:gluonnlp:Epoch: 0, Batch: 11399/88641, Loss=1.2425, lr=0.0000214 Time cost=4.1 Thoughput=12.26 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 11449/88641, Loss=1.1879, lr=0.0000215 Time cost=4.1 Thoughput=12.28 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 11499/88641, Loss=1.3414, lr=0.0000216 Time cost=4.1 Thoughput=12.29 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 11549/88641, Loss=1.2133, lr=0.0000217 Time cost=4.1 Thoughput=12.27 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 11599/88641, Loss=0.6397, lr=0.0000218 Time cost=4.1 Thoughput=12.21 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 11649/88641, Loss=1.3349, lr=0.0000219 Time cost=4.1 Thoughput=12.19 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 11699/88641, Loss=1.9567, lr=0.0000220 Time cost=4.1 Thoughput=12.20 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 11749/88641, Loss=0.9104, lr=0.0000221 Time cost=4.1 Thoughput=12.17 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 11799/88641, Loss=1.2024, lr=0.0000222 Time cost=4.1 Thoughput=12.19 samples/s
INFO:gluon

INFO:gluonnlp:Epoch: 0, Batch: 15149/88641, Loss=2.1788, lr=0.0000285 Time cost=4.1 Thoughput=12.26 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 15199/88641, Loss=1.1626, lr=0.0000286 Time cost=4.1 Thoughput=12.06 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 15249/88641, Loss=1.7322, lr=0.0000287 Time cost=4.2 Thoughput=12.02 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 15299/88641, Loss=1.3035, lr=0.0000288 Time cost=4.2 Thoughput=12.01 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 15349/88641, Loss=0.9746, lr=0.0000289 Time cost=4.2 Thoughput=12.00 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 15399/88641, Loss=1.1667, lr=0.0000290 Time cost=4.1 Thoughput=12.22 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 15449/88641, Loss=1.1696, lr=0.0000291 Time cost=4.1 Thoughput=12.26 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 15499/88641, Loss=1.2949, lr=0.0000291 Time cost=4.1 Thoughput=12.28 samples/s
INFO:gluonnlp:Epoch: 0, Batch: 15549/88641, Loss=1.4785, lr=0.0000292 Time cost=4.1 Thoughput=12.25 samples/s
INFO:gluon

...

INFO:gluonnlp:Epoch: 1, Batch: 65399/88641, Loss=2.3966, lr=0.0000234 Time cost=4.1 Thoughput=12.28 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 65449/88641, Loss=1.9131, lr=0.0000234 Time cost=4.1 Thoughput=12.29 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 65499/88641, Loss=1.6839, lr=0.0000234 Time cost=4.1 Thoughput=12.30 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 65549/88641, Loss=1.4222, lr=0.0000233 Time cost=4.1 Thoughput=12.29 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 65599/88641, Loss=0.9255, lr=0.0000233 Time cost=4.1 Thoughput=12.17 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 65649/88641, Loss=1.4318, lr=0.0000233 Time cost=4.2 Thoughput=11.97 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 65699/88641, Loss=2.0611, lr=0.0000233 Time cost=4.2 Thoughput=11.99 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 65749/88641, Loss=1.2486, lr=0.0000233 Time cost=4.2 Thoughput=12.02 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 65799/88641, Loss=1.5025, lr=0.0000233 Time cost=4.2 Thoughput=11.95 samples/s
INFO:gluon

INFO:gluonnlp:Epoch: 1, Batch: 69149/88641, Loss=1.6482, lr=0.0000226 Time cost=4.1 Thoughput=12.27 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 69199/88641, Loss=1.5573, lr=0.0000226 Time cost=4.1 Thoughput=12.15 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 69249/88641, Loss=2.0132, lr=0.0000226 Time cost=4.2 Thoughput=12.03 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 69299/88641, Loss=1.8906, lr=0.0000226 Time cost=4.1 Thoughput=12.27 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 69349/88641, Loss=1.8081, lr=0.0000225 Time cost=4.1 Thoughput=12.24 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 69399/88641, Loss=2.0015, lr=0.0000225 Time cost=4.1 Thoughput=12.24 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 69449/88641, Loss=1.1883, lr=0.0000225 Time cost=4.6 Thoughput=10.94 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 69499/88641, Loss=1.3868, lr=0.0000225 Time cost=4.1 Thoughput=12.21 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 69549/88641, Loss=1.7349, lr=0.0000225 Time cost=4.1 Thoughput=12.25 samples/s
INFO:gluon

INFO:gluonnlp:Epoch: 1, Batch: 72899/88641, Loss=1.5161, lr=0.0000218 Time cost=4.1 Thoughput=12.23 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 72949/88641, Loss=1.9171, lr=0.0000218 Time cost=4.1 Thoughput=12.25 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 72999/88641, Loss=1.3566, lr=0.0000218 Time cost=4.1 Thoughput=12.20 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 73049/88641, Loss=1.6701, lr=0.0000218 Time cost=4.1 Thoughput=12.15 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 73099/88641, Loss=1.6624, lr=0.0000218 Time cost=4.1 Thoughput=12.20 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 73149/88641, Loss=1.5490, lr=0.0000218 Time cost=4.1 Thoughput=12.14 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 73199/88641, Loss=1.1134, lr=0.0000217 Time cost=4.1 Thoughput=12.24 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 73249/88641, Loss=1.3910, lr=0.0000217 Time cost=4.1 Thoughput=12.24 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 73299/88641, Loss=1.6503, lr=0.0000217 Time cost=4.6 Thoughput=10.90 samples/s
INFO:gluon

INFO:gluonnlp:Epoch: 1, Batch: 76649/88641, Loss=1.2488, lr=0.0000210 Time cost=4.2 Thoughput=12.04 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 76699/88641, Loss=1.2278, lr=0.0000210 Time cost=4.5 Thoughput=11.20 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 76749/88641, Loss=1.2164, lr=0.0000210 Time cost=4.1 Thoughput=12.23 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 76799/88641, Loss=1.7373, lr=0.0000210 Time cost=4.1 Thoughput=12.24 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 76849/88641, Loss=1.8412, lr=0.0000210 Time cost=4.1 Thoughput=12.14 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 76899/88641, Loss=1.2768, lr=0.0000210 Time cost=4.2 Thoughput=12.00 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 76949/88641, Loss=1.5269, lr=0.0000210 Time cost=4.2 Thoughput=12.02 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 76999/88641, Loss=1.5091, lr=0.0000210 Time cost=4.1 Thoughput=12.24 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 77049/88641, Loss=1.3345, lr=0.0000209 Time cost=4.1 Thoughput=12.11 samples/s
INFO:gluon

INFO:gluonnlp:Epoch: 1, Batch: 80399/88641, Loss=1.2037, lr=0.0000202 Time cost=4.1 Thoughput=12.23 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 80449/88641, Loss=1.4740, lr=0.0000202 Time cost=4.1 Thoughput=12.19 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 80499/88641, Loss=1.7257, lr=0.0000202 Time cost=4.2 Thoughput=12.04 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 80549/88641, Loss=1.5663, lr=0.0000202 Time cost=4.1 Thoughput=12.23 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 80599/88641, Loss=1.6288, lr=0.0000202 Time cost=4.1 Thoughput=12.26 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 80649/88641, Loss=1.5895, lr=0.0000202 Time cost=4.1 Thoughput=12.25 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 80699/88641, Loss=1.7532, lr=0.0000202 Time cost=4.1 Thoughput=12.21 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 80749/88641, Loss=1.3047, lr=0.0000202 Time cost=4.1 Thoughput=12.22 samples/s
INFO:gluonnlp:Epoch: 1, Batch: 80799/88641, Loss=1.6022, lr=0.0000202 Time cost=4.3 Thoughput=11.70 samples/s
INFO:gluon

In [20]:
def evaluate():
    """Evaluate the model on validation dataset.
    """
    log.info('Loading dev data...')
#     if version_2:
#         dev_data = SQuAD('dev', version='2.0')
#     else:
    dev_data = SQuAD('dev', version='1.1')
    if args.debug:
        sampled_data = [dev_data[0], dev_data[1], dev_data[2]]
        dev_data = mx.gluon.data.SimpleDataset(sampled_data)
    log.info('Number of records in dev data:{}'.format(len(dev_data)))

    dev_dataset = dev_data.transform(
        SQuADTransform(
            copy.copy(tokenizer),
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_pad=False,
            is_training=False)._transform, lazy=False)
    
    dev_data_transform, _ = preprocess_dataset(
        dev_data, SQuADTransform(
            copy.copy(tokenizer),
            max_seq_length=max_seq_length,
            doc_stride=doc_stride,
            max_query_length=max_query_length,
            is_pad=False,
            is_training=False))
    log.info('The number of examples after preprocessing:{}'.format(
        len(dev_data_transform)))

    dev_dataloader = mx.gluon.data.DataLoader(
        dev_data_transform,
        batchify_fn=batchify_fn,
        num_workers=4, batch_size=test_batch_size,
        shuffle=False, last_batch='keep')

    log.info('start prediction')

    all_results = collections.defaultdict(list)

    epoch_tic = time.time()
    total_num = 0
    for data in dev_dataloader:
        example_ids, inputs, token_types, valid_length, _, _ = data
        total_num += len(inputs)
        out = net(inputs.astype('float32').as_in_context(ctx),
                  token_types.astype('float32').as_in_context(ctx),
                  valid_length.astype('float32').as_in_context(ctx))

        output = mx.nd.split(out, axis=2, num_outputs=2)
        example_ids = example_ids.asnumpy().tolist()
        pred_start = output[0].reshape((0, -3)).asnumpy()
        pred_end = output[1].reshape((0, -3)).asnumpy()

        for example_id, start, end in zip(example_ids, pred_start, pred_end):
            all_results[example_id].append(PredResult(start=start, end=end))
            
    epoch_toc = time.time()
    log.info('Time cost={:.2f} s, Thoughput={:.2f} samples/s'.format(
        epoch_toc - epoch_tic, total_num/(epoch_toc - epoch_tic)))

    log.info('Get prediction results...')

    all_predictions = collections.OrderedDict()

    for features in dev_dataset:
        results = all_results[features[0].example_id]
        example_qas_id = features[0].qas_id

        prediction, _ = predict(
            features=features,
            results=results,
            tokenizer=nlp.data.BERTBasicTokenizer(lower=lower),
            max_answer_length=max_answer_length,
            null_score_diff_threshold=null_score_diff_threshold,
            n_best_size=n_best_size,
#             version_2=version_2
        )

        all_predictions[example_qas_id] = prediction

    with io.open(os.path.join(output_dir, 'predictions.json'),
                 'w', encoding='utf-8') as fout:
        data = json.dumps(all_predictions, ensure_ascii=False)
        fout.write(data)

#     if version_2:
#         log.info('Please run evaluate-v2.0.py to get evaluation results for SQuAD 2.0')
#     else:
    F1_EM = get_F1_EM(dev_data, all_predictions)
    log.info(F1_EM)

In [21]:
evaluate()

INFO:gluonnlp:Loading dev data...


AttributeError: 'Namespace' object has no attribute 'debug'

## Deploy on SageMaker

1. Saving the model parameters
2. Preparing functions for inference 
3. Building a docker container with dependencies installed
4. Launch a serving end-point with SageMaker SDK

### 1. Save the model parameters

In [27]:
## save parameters, model definition and vocabulary in a zip file

# net.export('checkpoint')
with open('vocab.json', 'w') as f:
    f.write(vocab.to_json())
import tarfile
with tarfile.open("model.tar.gz", "w:gz") as tar:
    tar.add("output_dir/checkpoint-0000.params") 
    tar.add("output_dir/checkpoint-symbol.json") 
    tar.add("output_dir/vocab.json")

### 2. Preparing functions for inference

Two functions: 
1. model_fn() to load model parameters
2. transform_fn() to run model inference given an input

### 3. Building a docker container with dependencies installed

Let's prepare a docker container with all the dependencies required for model inference. Here we build a docker container based on the SageMaker MXNet inference container, and you can find the list of all available inference containers at https://docs.aws.amazon.com/sagemaker/latest/dg/pre-built-containers-frameworks-deep-learning.html

Here we use local mode for demonstration purpose. To deploy on actual instances, you need to login into AWS elastic container registry (ECR) service, and push the container to ECR. 

```
docker build -t $YOUR_EDR_DOCKER_TAG . -f Dockerfile
$(aws ecr get-login --no-include-email --region $YOUR_REGION)
docker push $YOUR_EDR_DOCKER_TAG
```

In [42]:
%%writefile Dockerfile

ARG REGION
FROM 763104351884.dkr.ecr.$REGION.amazonaws.com/mxnet-inference:1.6.0-gpu-py3

RUN pip install --upgrade --user --pre 'mxnet-mkl' 'https://github.com/dmlc/gluon-nlp/tarball/v0.9.x'

RUN pip list | grep mxnet

COPY *.py /opt/ml/model/code/

Overwriting Dockerfile


In [58]:
## Docker login cmd
!$(aws ecr get-login --no-include-email --region us-east-1 --registry-ids 763104351884)

WARNING! Using --password via the CLI is insecure. Use --password-stdin.
WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [60]:
!export REGION=$(wget -qO- http://169.254.169.254/latest/meta-data/placement/availability-zone) &&\
 docker build --no-cache --build-arg REGION=${REGION::-1} -t my-docker:inference . -f Dockerfile

Sending build context to Docker daemon  2.127GB
Step 1/5 : ARG REGION
Step 2/5 : FROM 763104351884.dkr.ecr.$REGION.amazonaws.com/mxnet-inference:1.6.0-gpu-py3
1.6.0-gpu-py3: Pulling from mxnet-inference

7927d38a: Pulling fs layer 
ac894db4: Pulling fs layer 
2af6d627: Pulling fs layer 
86211d23: Pulling fs layer 
603ff777: Pulling fs layer 
7165632f: Pulling fs layer 
96e40dcf: Pulling fs layer 
91ff3706: Pulling fs layer 
02a4385b: Pulling fs layer 
e229cfdb: Pulling fs layer 
0e6ed5b5: Pulling fs layer 
c8e328fe: Pulling fs layer 
bb20abb1: Pulling fs layer 
0702cb67: Pulling fs layer 
d6c2671b: Pulling fs layer 
486e676d: Pulling fs layer 
a8b75933: Pulling fs layer 
7d871d5a: Pulling fs layer 
c8e48618: Pulling fs layer 
9ef7425f: Pulling fs layer 
c02fa024: Pulling fs layer 
0702cb67: Downloading  686.7MB/686.7MBKDownloading  538.1kB/270.4MBwrite /home/ec2-user/SageMaker/docker-data/tmp/GetImageBlob331553077: no space left on device


In [30]:
!wget -qO- http://169.254.169.254/latest/meta-data/placement/availability-zone

us-east-1a

In [56]:
# !aws ecr get-login --no-include-email --registry-ids 763104351884
!aws ecr get-login --no-include-email --region us-east-1 --registry-ids 763104351884

docker login -u AWS -p eyJwYXlsb2FkIjoiV3FDZFFmeFhrOXZ2aVhsSzZ4YUFmU2U0aWJsRHp5ZEJLRjhNd3hXcW5KREwveEtDcjdUaUhRZlJ0NEg2ZmR2ZWswcVQ2eGFjMUswVWFtblhwOWNTVUROUkloRWZaL1M3YWU1K01JZEdUbFAvMmtWV09nWW1LUExMVFArV2JPblhxR0dSVGFFQjA1RHdrcGFEcThtVFFidjU0WWtpRkpqRkttRzAvVzNUTWxUM3VqRWp3ZkppVWlIMnBZU3JYY3lqWHBtUlRsUXNJbzRRbDJkVG5TM25FVkRlRm5jckFYOHpZUWNtY2RtcmxqSHo3UEhSZ2h5RVcxWjI3eU9Yd2FGckVDRTk3VXhYVy9aVHB0N2RSb21HVERCVGcrRVJ1RDVCb0hyVGF0SGk3RC9ROEs2d0ZaUVE0YXYrT1NQbUJQSUViNFFBOSswMG5tWXR4Rmk3K3c2eU8vSjl6eXdtRW9nQzA3ejdiTHpIRjduZDRTeHlVK2hzTjJkV1ZjWVBpVXI3aU05d21ESEdJZWkxWkJjWnk5eW9CWGNwcE00OXhGb0NBRE80b3J3L1NRNmpxdk5NSi9sU1FYb01yNU1GV0d2Sy9QdTNXUnlXcEpobjU5WDUrOEsxalBxWnhuTVJrTXdxb1A5N2RYQzdZUFF6NCtFYXVvNSszMHROck9QZktiMmQyYk9US1htOHhIa2NsblJhVm84aVlpaWl3QnhVSWJHalhIOHB0eVVrSVUzTlo5Nkw5TVpTc0YzdmdhTzZVQytPSFFXbTdsQU9obk15Wm1GeVVoQlA2OTFDNHJlY1lGTEIxc1IwZVJtQzhRMXRZMW45NThzMDhCK2F6VVlHYWZtbE9NSjdraStjMTB6SnA1WXhseGRsejdIdGhicVE4WHJrVkJGSHFJaWszRjVuRERBMWd0NGZQdHIzTnBkWnp4YjZjVzRsZ0lQTjBoQVgxYlhsV

In [59]:
!docker pull 763104351884.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.6.0-gpu-py3
#              763104351884.dkr.ecr.<region>.amazonaws.com/mxnet-inference:1.4.1-gpu-py3

1.6.0-gpu-py3: Pulling from mxnet-inference

7927d38a: Pulling fs layer 
ac894db4: Pulling fs layer 
2af6d627: Pulling fs layer 
86211d23: Pulling fs layer 
603ff777: Pulling fs layer 
7165632f: Pulling fs layer 
96e40dcf: Pulling fs layer 
91ff3706: Pulling fs layer 
02a4385b: Pulling fs layer 
e229cfdb: Pulling fs layer 
0e6ed5b5: Pulling fs layer 
c8e328fe: Pulling fs layer 
bb20abb1: Pulling fs layer 
0702cb67: Pulling fs layer 
d6c2671b: Pulling fs layer 
486e676d: Pulling fs layer 
a8b75933: Pulling fs layer 
165632f: Waiting fs layer 
c8e48618: Pulling fs layer 
2a4385b: Waiting fs layer 
c02fa024: Pulling fs layer 
486e676d: Download complete  B/768.9MB0ADownloading  124.5MB/688.7MBDownloading  377.8MB/768.9MBwrite /home/ec2-user/SageMaker/docker-data/tmp/GetImageBlob671858852: no space left on device


In [32]:
!aws ecr get-login --no-include-email --region us-east-1

docker login -u AWS -p eyJwYXlsb2FkIjoidWdBRkZyVERaMGFNRnJzYUFKNHZZQm1oZ0I5eVFJd25qRDBpcit2VU9SU3EwWkg1RTEyaVVTdWJSSmpJMmpzbXVjVnZxYzZVR01CcVpaa0JKb2oreEMycCtMNytWK2ZhNHh3bVlLMnZVWnFWN1ByYVp4Y0FaYjlaSHpWRFpXYXlabHZ4a0NQamVDbE55a3JpSkNrMGNwLzR5TU9ONnRTTHpRWEdGOTFpQWRldzE2Y05SbWtpWXZsVGZZWkJVSjJTemF5MEFLTXcwMHBzZmt5UXM0VkYycTlDcW1oRDZTMmVQWEY3UjNDRjUzSnN2cHpBbDRTRHorQ0dIOEUyaUNETlZobUVCQUpTNFZLR3E4ZGd6MWFGRjRhZkJVc01PUnE4Tm9zc3pvQ3J0b2dsS2o4Qkx4MHc4VXZySUNWODFjNWJnTnM3YlJTUVQzQ2s2enY1NERtRXRDZEk5UEZkdmwrTHZHQjNKTTEzZlQ0UUtmOUU0Y214TzY3ekpwUUNmSHcwTmN5VzQ3bVdVRnJxajhiaTJBNk54SlltRU12K2xQNjVjbHI1UGVZTTBvZTlZc2hDaGJxT2xlOWVuMTBTZlJmMG1DYmNDcDdDRUtaMUEyTUs2a0t5dEJoQWVrN1Y0U0hGRmJKdHJIa01Hb2ZNTGk2L2tmS2QzZ2lySy8weWgxT2VFcnlJaXBNL0Q5MFE0UTBxbHBXSktBVk5BTlhYTFdLVDd2Q2J5OXJOZjJ2aktpRWVuRFo0bUt3YWFSV3YwcmMvT0h1M1BJWFh6ZWJ4QitONVVWM251emgyc3Y3RTFXaWdyeWpYcTUxS1pqUmR4ck9XK2Z4OU94VlM2SzVyU081WVZ6cE9zRVkzMkFKcHdyYlRCblRBZTU0TUJBdWduczRhM3BOdEVQU3E2OU5PYW0yc0RGV2tVTitvL2F0M1dMRUduaURIYmg2c2NTNlZrbXAvd

In [45]:
!docker pull 520713654638.dkr.ecr.us-east-1.amazonaws.com/mxnet-inference:1.6.0-gpu-py3


Error response from daemon: Get https://520713654638.dkr.ecr.us-east-1.amazonaws.com/v2/mxnet-inference/manifests/1.6.0-gpu-py3: no basic auth credentials


In [34]:
import sagemaker

role=sagemaker.get_execution_role()


In [38]:
!aws ecr get-login --no-include-email --registry-ids 520713654638

docker login -u AWS -p eyJwYXlsb2FkIjoiV2Z4Q05ESDd3WW5hYXpVZjc2c3JrM3RBSTZQZUV6NGw2VFRrQ0dLd2E0dVNQTjdkSkdNSFk4Y2dNcGoxcXcrN25ZY0lLcDN4aC9IMzhZb2tqUU1WWld2WGlYc2lhbFdHelppMVB3b3ErcUVnWXVybUJxUjhuZ2ZzcTRnQmtRRFYrZXI1UDJxRExrUHFtVlJ3U2w4eERScFFZVXIwbzF6MGtOQ0dVdXpocDVSZmlXc0hHY29qeXdsaFEwWEpMYkczMGhhUUJGZUVYekRpdEJLL1Ezc0tYUm90djlyRlVLSHdhcG5UOEEzL3JsRDZxNEp6TjFORDI5RlRoVE9QY0g4YXRidExISG9LL3c3RTNXTTNVZERQbTk5NmNsa0hVWFpjM2tFeDhzS1lUM3Z1QzF6T1J1TS9RaGJxdE91Y29ZSXUwaXBPZnJIYUdLRXc3aFBKYktHb1RUR0Z0b0xwZVZ1UWtDV0w1YjVsVDk3VkxJZEU4REZsTkw1bzF0VzU2NWptb3dYc2RpTk5MT3gvdkhtbmFHa0YzVXlnL01EQkcyWUxYMTVqYThiNWV3MlFMUEEzMUJsNkpTQUxaMEhzUFp1M01qQk5XTFN2eXIyeGw2anNZZy9jRzVleVNvbUZGbUo1SVI0WGFXMmZLWnIySDQrYUt4TVk5ODVENUZKYzZ4M3lKNUhHa3JzbVVFb2xXNFd2MG9VVEFSUVVvSTdnQ3VVaUtCRWlvZkpPRk9ublc2L1MxOXgyQ3hOT2d3aDVpTi81MU4zVnEwRlhtZGwwNmlheFB0MWk4cmVkbndNM2Y5R0tMemtSUnR6bTlQTFJNbjFXMTVDVHBwNExSMS9QajVKQzZoTXYrRlJKcXpKenVWbll4dS9rMW5UM0Q2YVAySEIwR0dDLzBEOW5VQUhwRkdic2V5YlJQSXJSMXF1b3lHd1pDMERsMXdLcTdvcVpIK1ZjU